![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [26]:
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

# Caminhos dos arquivos
filename_features = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\features.txt"
filename_labels = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\activity_labels.txt"

filename_subtrain = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\train\subject_train.txt"
filename_xtrain = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\train\X_train.txt"
filename_ytrain = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\train\y_train.txt"

filename_subtest = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\test\subject_test.txt"
filename_xtest = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\test\X_test.txt"
filename_ytest = r"C:\Users\Joel\Downloads\Data Scientist\Mod 27\M27 E M29\M27 E M29\test\y_test.txt"

# Carregando os dados
features = pd.read_csv(filename_features, header=None, delim_whitespace=True, names=['index', 'nome_var'])
features = features['nome_var']  # Seleciona apenas o nome das variáveis

# Adicionando um índice único para duplicatas
if features.duplicated().any():
    print("Detectadas colunas duplicadas. Renomeando-as para serem únicas.")
    features = features + "_" + features.groupby(features).cumcount().astype(str)

# Carregando os rótulos das atividades
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

# Dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label']).squeeze()

# Dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label']).squeeze()

# Verificando os dados
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Nomes das colunas únicas? {'Sim' if X_train.columns.is_unique else 'Não'}")

Detectadas colunas duplicadas. Renomeando-as para serem únicas.
Shape de X_train: (7352, 561)
Shape de X_test: (2947, 561)
Nomes das colunas únicas? Sim


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [22]:
%%time

# Criando e avaliando o modelo
start_time = time.time()

clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf.fit(X_train, y_train)

train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, clf.predict(X_test))

end_time = time.time()

# Resultados
print(f"Acurácia na base de treinamento: {train_accuracy:.4f}")
print(f"Acurácia na base de teste: {test_accuracy:.4f}")
print(f"Tempo de processamento: {end_time - start_time:.2f} segundos")


Acurácia na base de treinamento: 0.9758
Acurácia na base de teste: 0.8799
Tempo de processamento: 4.82 segundos
CPU times: total: 4.83 s
Wall time: 4.82 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [28]:
%%time
# Normalização das variáveis
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA (somente 1 componente)
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Criando e avaliando o modelo
start_time = time.time()

clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
clf.fit(X_train_pca, y_train)

train_accuracy = accuracy_score(y_train, clf.predict(X_train_pca))
test_accuracy = accuracy_score(y_test, clf.predict(X_test_pca))

end_time = time.time()

# Resultados
print(f"Acurácia na base de treinamento: {train_accuracy:.4f}")
print(f"Acurácia na base de teste: {test_accuracy:.4f}")
print(f"Tempo de processamento: {end_time - start_time:.2f} segundos")

Acurácia na base de treinamento: 0.4771
Acurácia na base de teste: 0.4316
Tempo de processamento: 0.05 segundos
CPU times: total: 609 ms
Wall time: 378 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [30]:
%%time
# Lista de quantidades de componentes principais a serem testadas
components_list = [1, 2, 5, 10, 50]

# Avaliação do modelo para cada quantidade de componentes
results = []

for n_components in components_list:
    print(f"Testando com {n_components} componentes...")
    
    # PCA com o número especificado de componentes
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)
    
    # Tempo de treinamento e avaliação
    start_time = time.time()
    
    # Criando e treinando a árvore de decisão
    clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
    clf.fit(X_train_pca, y_train)
    
    train_accuracy = accuracy_score(y_train, clf.predict(X_train_pca))
    test_accuracy = accuracy_score(y_test, clf.predict(X_test_pca))
    
    end_time = time.time()
    
    # Salvando os resultados
    results.append({
        "n_components": n_components,
        "train_accuracy": train_accuracy,
        "test_accuracy": test_accuracy,
        "processing_time": end_time - start_time
    })

# Exibindo os resultados
results_df = pd.DataFrame(results)
print(results_df)

Testando com 1 componentes...
Testando com 2 componentes...
Testando com 5 componentes...
Testando com 10 componentes...
Testando com 50 componentes...
   n_components  train_accuracy  test_accuracy  processing_time
0             1        0.477149       0.431625         0.052858
1             2        0.590180       0.546318         0.054852
2             5        0.832699       0.770614         0.069809
3            10        0.858678       0.767221         0.128737
4            50        0.888194       0.775365         0.577270
CPU times: total: 3.61 s
Wall time: 1.26 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

<span style="color:red;">Quanto maior o número de componentes é melhor a acurácia e o seu tempo de processamento é maior também.</span>